In [ ]:
# %pip install hyperopt

In [23]:
#import packages and soil moisture water balance function
%reload_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import glob
import geopandas as gpd
import os
import pandas as pd
import sys
sys.path.insert(0, '../../PixSWAB_v1')
import calibration_tool.pixswab_v1_for_calibration as pixswab
import model.hydroStats as hs
import datetime
import time
from deap import algorithms
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

## Set global parameter
global gen
gen = 0
WarmupDays = 0

import warnings
warnings.filterwarnings('ignore')


# Pixswab Optimization

In [11]:
# start a cleint for distributed computing for files larger than the memory capacity (RAM)
# from dask.distributed import Client, LocalCluster
# import multiprocessing as mp
# from dask.distributed import progress

# # multiprocessing client
# client = pixswab.start_multiprocessing()
# client.restart()

In [12]:
# get the config file and prepare inputs
config_path = r'..\config_files\pixswab_config.ini'
pixswab_inputs, pixswab_parameters, log_file_path, out_dir, encoding_output, bsahpe = pixswab.prepare_inputs(config_path)

In [13]:
# read observed ourflw (discharge)
Qobs_file = r'..\..\Upper_Litani\input\observed_flow\discharge_Upper_Litani_JoubJannine_beforeQaraoun.csv'
Qobs=pd.read_csv(Qobs_file,sep=',',index_col=0)
target_column = 'km3/month'
Qobs = Qobs[target_column]
Qobs.index = pd.to_datetime(Qobs.index)


# DEAP

In [29]:
# parameter_range
param_range_file = r'..\config_files\ParamRanges.csv'
ParamRanges=pd.read_csv(param_range_file,sep=',',index_col=0)
# print(ParamRanges)

def RunModel(Individual):
    # Convert scaled parameter values ranging from 0 to 1 to usncaled parameter values
    Parameters = [None] * len(ParamRanges)
    for ii in range(0,len(ParamRanges-1)):
        Parameters[ii] = Individual[ii]*(float(ParamRanges.iloc[ii,1])-float(ParamRanges.iloc[ii,0]))+float(ParamRanges.iloc[ii,0])
    # print(Parameters)
    result = pixswab.run_pixswab(pixswab_inputs, Parameters)
    Qsim = pixswab.compute_discharge(result, bsahpe, out_dir, log_file_path)
    Qsim_target_column = 'discharge'
    Qsim = Qsim[Qsim_target_column]
    Qsim = Qsim[Qsim.index.isin(Qobs.index)]
    KGE = hs.KGE(s=Qsim, o=Qobs, warmup=WarmupDays)
    return KGE,
    

In [ ]:
########################################################################
#   Perform calibration using the DEAP module
########################################################################

# firstrun = parser.getboolean('Option', 'firstrun')   # using default run as first run
# if firstrun:
#     para_first = ast.literal_eval(parser.get("Option", "para_first"))
# bestrun = parser.getboolean('Option', 'bestrun')
import array
import random
import multiprocessing
import pickle

path_calibration_results = r'..\..\Upper_Litani\calibration_results'

maximize = True
use_multiprocessing = 0
pool_limit = 32
ngen = 10
mu = 16
lambda_ = 8

firstrun = True
para_first = [0.5, 0.5, 2.5, 1.]
# pref. flow, groundwater recession, No of run
bestrun = True


# first standard parameter set
# recalculated to a population setting
if firstrun:
    #para_first = [0.19, 0.53, 6.]
    para_first2 = []
    for ii in range(0, len(ParamRanges - 1)):
        delta = float(ParamRanges.iloc[ii, 1]) - float(ParamRanges.iloc[ii, 0])
        if delta == 0:
            para_first2.append(0.)
        else:
            para_first2.append((para_first[ii] - float(ParamRanges.iloc[ii, 0])) / delta)

ii = 1


if maximize: maxDeap = 1.0
else: maxDeap = -1.0

creator.create("FitnessMin", base.Fitness, weights=(maxDeap,))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_float", random.uniform, 0, 1)

# Structure initializers
toolbox.register("Individual", tools.initRepeat, creator.Individual, toolbox.attr_float, len(ParamRanges))
toolbox.register("population", tools.initRepeat, list, toolbox.Individual)

def checkBounds(min, max):
    def decorator(func):
        def wrappper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrappper
    return decorator

toolbox.register("evaluate", RunModel)
toolbox.register("mate", tools.cxBlend, alpha=0.15)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.3, indpb=0.3)
toolbox.register("select", tools.selNSGA2)

toolbox.decorate("mate", checkBounds(0, 1))
toolbox.decorate("mutate", checkBounds(0, 1))

# if __name__ == "__main__":

t = time.time()

if use_multiprocessing==True:
    pool_size = multiprocessing.cpu_count() * 1
    print(pool_size, pool_limit)
    if pool_size > pool_limit: pool_size = pool_limit
    pool = multiprocessing.Pool(processes=pool_size)
    toolbox.register("map", pool.map)
    print('pool_size:', pool_size)


# For someone reason, if sum of cxpb and mutpb is not one, a lot less Pareto optimal solutions are produced
cxpb = 0.7
mutpb = 0.3

startlater = False
checkpoint = os.path.join(path_calibration_results,"checkpoint.pkl")
# checkpoint = r'..\..\Upper_Litani\calibration_results\checkpoint.pkl'
if os.path.exists(os.path.join(checkpoint)):
    with open(checkpoint, "rb" ) as cp_file:
        cp = pickle.load(cp_file)
        population = cp["population"]
        start_gen = cp["generation"]
        random.setstate(cp["rndstate"])
        if start_gen > 0:
            offspring = cp["offspring"]
            halloffame =  cp["halloffame"]
            startlater = True
            gen = start_gen

else:
    population = toolbox.population(n=mu)
    # Numbering of runs
    for ii in range(mu):
        population[ii][-1]= float(gen * 1000 + ii+1)
    #first run parameter set:
    if firstrun:
        for ii in range(len(population[0])):
            population[0][ii] = para_first2[ii]
        population[0][-1] = 0.

effmax = np.zeros(shape=(ngen+1,1))*np.NaN
effmin = np.zeros(shape=(ngen+1,1))*np.NaN
effavg = np.zeros(shape=(ngen+1,1))*np.NaN
effstd = np.zeros(shape=(ngen+1,1))*np.NaN
if startlater == False:
    halloffame = tools.ParetoFront()

    # saving population
    cp = dict(population=population, generation=gen, rndstate=random.getstate())
    with open(checkpoint, "wb") as cp_file:
        pickle.dump(cp, cp_file)
    cp_file.close()

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    halloffame.update(population)
    # print('here 1-3')
    # Loop through the different objective functions and calculate some statistics
    # from the Pareto optimal population

    for ii in range(1):
        effmax[0,ii] = np.amax([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
        effmin[0,ii] = np.amin([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
        effavg[0,ii] = np.average([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
        effstd[0,ii] = np.std([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
    gen = 0
    print(">> gen: "+str(gen)+", effmax_KGE: "+"{0:.3f}".format(effmax[gen,0]))
    gen = 1


# Begin the generational process
conditions = {"ngen" : False, "StallFit" : False}
while not any(conditions.values()):
    if startlater == False:
        # Vary the population
        offspring = algorithms.varOr(population, toolbox, lambda_, cxpb, mutpb)

        # put in the number of run
        for ii in range(lambda_):
            offspring[ii][-1] = float(gen * 1000 + ii + 1)

    # saving population
    cp = dict(population=population, generation=gen, rndstate=random.getstate(), offspring=offspring, halloffame=halloffame)
    with open(checkpoint, "wb") as cp_file:
        pickle.dump(cp, cp_file)
    cp_file.close()
    startlater = False



    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    if halloffame is not None:
        halloffame.update(offspring)

    # Select the next generation population
    population[:] = toolbox.select(population + offspring, mu)

    # put in the number of run
    #for ii in xrange(mu):
    #    population[ii][-1] = float(gen * 1000 + ii + 1)

    # Loop through the different objective functions and calculate some statistics
    # from the Pareto optimal population
    for ii in range(1):
        effmax[gen,ii] = np.amax([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
        effmin[gen,ii] = np.amin([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
        effavg[gen,ii] = np.average([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
        effstd[gen,ii] = np.std([halloffame[x].fitness.values[ii] for x in range(len(halloffame))])
    print(">> gen: "+str(gen)+", effmax_KGE: "+"{0:.3f}".format(effmax[gen,0]))

    # Terminate the optimization after ngen generations
    if gen >= ngen:
        print(">> Termination criterion ngen fulfilled.")
        conditions["ngen"] = True

    gen += 1

# Finito
if use_multiprocessing == True:
    pool.close()
elapsed = time.time() - t
print(">> Time elapsed: "+"{0:.2f}".format(elapsed)+" s")

########################################################################
#   Save calibration results
########################################################################

# Save history of the change in objective function scores during calibration to csv file
print(">> Saving optimization history (front_history.csv)")
front_history = pd.DataFrame({'gen':list(range(gen)),
                                  'effmax_R':effmax[:,0],
                                  'effmin_R':effmin[:,0],
                                  'effstd_R':effstd[:,0],
                                  'effavg_R':effavg[:,0],
                                  })
front_history.to_csv(os.path.join(path_calibration_results,"front_history.csv"),',')
# as numpy  numpy.asarray  ; numpy.savetxt("foo.csv", a, delimiter=","); a.tofile('foo.csv',sep=',',format='%10.5f')

# Compute overall efficiency scores from the objective function scores for the
# solutions in the Pareto optimal front
# The overall efficiency reflects the proximity to R = 1, NSlog = 1, and B = 0 %
front = np.array([ind.fitness.values for ind in halloffame])
effover = 1 - np.sqrt((1-front[:,0]) ** 2)
best = np.argmax(effover)

# Convert the scaled parameter values of halloffame ranging from 0 to 1 to unscaled parameter values
paramvals = np.zeros(shape=(len(halloffame),len(halloffame[0])))
paramvals[:] = np.NaN
for kk in range(len(halloffame)):
    for ii in range(len(ParamRanges)):
        paramvals[kk][ii] = halloffame[kk][ii]*(float(ParamRanges.iloc[ii,1])-float(ParamRanges.iloc[ii,0]))+float(ParamRanges.iloc[ii,0])

# Save Pareto optimal solutions to csv file
# The table is sorted by overall efficiency score
print(">> Saving Pareto optimal solutions (pareto_front.csv)")
ind = np.argsort(effover)[::-1]
pareto_front = pd.DataFrame({'effover':effover[ind],'R':front[ind,0]})
for ii in range(len(ParamRanges)):
    pareto_front["param_"+str(ii).zfill(2)+"_"+ParamRanges.index[ii]] = paramvals[ind,ii]
pareto_front.to_csv(os.path.join(path_calibration_results,"pareto_front.csv"),',')

# Select the "best" parameter set and run Model for the entire forcing period
Parameters = paramvals[best,:]


In [ ]:
Parameters

In [53]:
best_error

-0.8139443541372209